# Homework: Gaussian Quadrature Integration

###### GH: Gauss-Hermite; GL: Gauss-Legendre; GLa: Gauss-Laguerre

For all the problems, you may obtain $\{\omega_i, \xi_i \}$ directly from Julia's `FastGaussQuadrature` packages. For instance, `xi, wi = gausslegendre(20)` would give $\{\omega_i, \xi_i \}$, $i=1,\ldots,20$, from the Gauss-Legendre rule.

## This problem asks you to adopt the Gauss-Hermite rule to an infinite integral where the weight function does not match.  The integration problem is:
  $$\begin{aligned}
   I = \int_{-\infty}^\infty g(x) d x =  \int_{-\infty}^\infty \exp\left(-\frac{1}{3}x^2\right)\sqrt{1+x^2} dx.
  \end{aligned}$$ 

- ##### Let's do this without using changes of variables. 
  - Transform $g(x)$ to $e^{-x^2}f(x)$ so that the Gauss-Hermite rule could be applied. Write down the re-formulated integration problem explicitly and write down $f(x)$ explicitly.
  - Write Julia code to integrate the function numerically.

Transform $g(x)$ to $e^{-x^2}f(x)$ so we can apply GH rule directly without changes of variables:

$$
I = \int_{-\infty}^\infty \exp\left(-\frac{1}{3}x^2\right)\sqrt{1+x^2} dx = \int_{-\infty}^\infty e^{-x^2} \cdot e^{x^2} \exp\left(-\frac{1}{3}x^2\right)\sqrt{1+x^2} dx
$$

so $f(x)$ should be like:

$$
f(x) = e^{x^2} \exp\left(-\frac{1}{3}x^2\right)\sqrt{1+x^2}
$$

In [41]:
using FastGaussQuadrature

f(x) = exp(x^2) * exp((-x^2) / 3) * sqrt(1 + x^2)
xi, wi = gausshermite(30)
sum(@. f(xi) * wi)

4.5638130088853694

## Write a Julia function with the following specifications.
  - It calculates $E[g(y)]$ for any arbitrary $g(y)$ where $y \sim N(\mu, \sigma^2)$.
  - User's inputs include $g(y)$, $\mu$, $\sigma^2$, and $n$.
  - You may obtain $\{\omega_i, \xi_i\}$ directly from Julia's package.
  - If $g(y) = 1 + 3y + y^2$, $y \sim N(1,2)$, and $n=30$, the interface could be like `myInteg(1 + 3y + y^2, mean=1, variance=2, n=30)`, or, `myInteg(g, mean=1, variance=2, n=30)`.
  - Hint: You may define the probability density function of $y$ using the `Distributions` package and then `pdf(Normal(mean, std.dev.),x)`.

In [42]:
function myInteg(g, μ, σ², n)
    f(t) = (1 / sqrt(π)) * g(sqrt(2 * σ²) * t + μ)
    xi, wi = gausshermite(n)
    return sum(@. f(xi) * wi)
end

#= Approximation Sample =#
g(t) = 1 + 3 * t + t^2
myInteg(g, 1.0, 2.0, 30)

6.999999999999995

## It is generally true that proper integrals (i.e., with finite domains) are numerically easier to deal with. But what if a problem is naturally a GH type with an infinite domain and we transform it to a GL problem with a finite domain? Is the GL rule better in this scenario? Let's find out the answer. Consider the following integration problem which is often encountered in econometric analysis. 

$$\begin{aligned}
      I = \int_{-\infty}^\infty \Phi\left(\frac{x-a}{b}\right)\phi(x) dx,
\end{aligned}$$

where $X\sim N(0,1)$ is a standard normal random variable, $\phi(z)$ is the density function (PDF) of a random variable $z$, and $\Phi(z)$ is the CDF of $z$.
   - ##### Write a Julia function that uses the GH rule to approximate the integral. The program should be a function of $a$ and $b$.
   - ##### Transform this problem to the GL type. What is the transformation rule and what is the Jacobian? Type the equations and show your work.
   - ##### Write a Julia function that uses the GL rule to approximate the integral. The program should be a function of $a$ and $b$.
   - ##### Suppose the required precision is 6 digits after the decimal. Compare the performances of these functions. Which one converges more quickly (smaller $n$)? You may pick values of $a$ and $b$ to do the comparison. You may draw graphs to compare the results.
   - ##### Based on your results, is the GL rule always the most efficient?

For transforming the problem to Gauss-Legendre type, let

$$\begin{aligned}
x(t) = \frac{t}{1 - t^2}
\end{aligned}$$

Hence, the Jacobian is

$$\begin{aligned}
Jacobian(t) = \frac{t^2 + 1}{(t^2 -1)^2}
\end{aligned}$$

In [39]:
using FastGaussQuadrature, Distributions

#= Uses GH rule to approximate the integral =#
function GH_approximation1(a, b; n=30, d=Normal(0, 1))
    f(x) = cdf(d, (x - a) / b) * pdf(d, x) * exp(x^2)
    xi, wi = gausshermite(n)
    return sum(@. f(xi) * wi)
end

#= Uses GL rule to approximate the integral =#
function GL_approximation1(a, b; n=30, d=Normal(0, 1))
    jacobian(t) = (t^2 + 1) / (t^2 - 1)^2    # jacobian
    x(t) = t / (1 - t^2)                     # transformation: x -> t
    f(t) = cdf(d, (x(t) - a) / b) * pdf(d, x(t)) * jacobian(t)
    xi, wi = gausslegendre(n)
    return sum(@. f(xi) * wi)
end

#= approximation sample =#
GH_approximation1(3, 15, n=70, d=Normal(0, 1)) |> display
GL_approximation1(3, 15, n=70, d=Normal(0, 1)) |> display

0.4209135177962354

0.4209135176177315

We can see from the graph below that Gauss-Hermite converges more quickly in this scenario, so Gauss-Legendre is not always the most efficient.

In [33]:
using Interact, WebIO, Plots, LinearAlgebra, Statistics
@manipulate for a=-100:1.5:100, b=-100:1.5:100
    GH_res = zeros(size(3:53)[1])
    GL_res = zeros(size(3:53)[1])
    
    for n=3:53
        GH_res[n-2] = GH_approximation1(a, b, n=n, d=Normal(0, 1))
        GL_res[n-2] = GL_approximation1(a, b, n=n, d=Normal(0, 1))
    end
        
    plot(collect(3:53), [GH_res, GL_res], xlabel="number of nodes", ylabel="approximation", label=["Gauss-Hermite" "Gauss-Legendre"])
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["a"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 134, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable{Any}(67), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/chriscyhxx/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/chriscyhxx/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/chriscyhxx/.julia/packages/InteractBase/Qhvxg/src/../assets/all.js"), Asset("css", nothing, "/Users/chriscyhxx/.julia/packages/InteractBase/Qhvxg/src/../assets/style.css"), Asset("css", nothing, "/Users/chriscyhxx/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000177f1ca90, Task (runnable) @0x0000000177f1ca90), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"-100.0\",\"-98.5\",\"-97.0\",\"-95.5\",\"-94.0\",\"-92.5\",\"-91.0\",\"-89.5\",\"-88.0\",\"-86.5\",\"-85.0\",\"-83.5\",\"-82.0\",\"-80.5\",\"-79.0\",\"-77.5\",\"-76.0\",\"-74.5\",\"-73.0\",\"-71.5\",\"-70.0\",\"-68.5\",\"-67.0\",\"-65.5\",\"-64.0\",\"-62.5\",\"-61.0\",\"-59.5\",\"-58.0\",\"-56.5\",\"-55.0\",\"-53.5

## This problem asks you to consider an integration problem where it can be transformed to different types to use different Gaussian quadrature rules. Which one is the best? The integration problem is:

 $$\begin{aligned}
 \int_{a}^\infty \exp\left(-\frac{1}{100}x^2\right)(1+x^2) dx,
 \end{aligned}$$

where $a$ is finite but not necessarily 0. This problem has some characteristics of the GLa and GH types: It has an exponential function, and by mapping $a$ to $-1$, $0$ or $-\infty$ via change of variables, the domain would be of the GL, GLa and GH types, respectively. You are asked to perform the approximation using rules of each of the types and compare the performance. In the code, make $a$ a parameter that we can assign a value to it.

 - ##### Write Julia code that uses the GL rule to approximate the integral. 
 - ##### Write Julia code that uses the GLa rule to approximate the integral. 
 - ##### Write Julia code that uses the GH rule to approximate the integral. 
 - ##### Suppose the required precision is 6 digits after the decimal. Compare the performances of these functions; drawing a graph will be useful. You may pick a value for $a$ in the comparison. 
 - ##### According to your result above, which rule converges more quickly (smaller $n$)? What is the intuition behind the result?

In [35]:
function GL_approximation2(a; n=30)
    x(t) = a + (1 + t) / (1 - t)
    jacobian(t) = 2 / (t - 1)^2
    f(t) = exp((-1/100) * x(t)^2) * (1 + x(t)^2) * jacobian(t)
    xi, wi = gausslegendre(n)
    return sum(@. f(xi) * wi)
end

function GLa_approximation2(a; n=30)
    x(t) = t + a
    jacobian(t) = 1
    f(t) = exp((-1/100) * x(t)^2) * (1 + x(t)^2) * jacobian(t) * exp(t)
    xi, wi = gausslaguerre(n)
    return sum(@. f(xi) * wi)
end

function GH_approximation2(a; n=30)
    x(t) = a + exp(t)
    jacobian(t) = exp(t)
    f(t) = exp((-1/100) * x(t)^2) * (1 + x(t)^2) * jacobian(t) * exp(t^2)
    xi, wi = gausshermite(n)
    return sum(@. f(xi) * wi)
end

#= approximation sample =#
GL_approximation2(1, n=100)  |> display
GLa_approximation2(1, n=100) |> display
GH_approximation2(1, n=100)  |> display

450.64771532155316

450.6477148802903

450.64770638010737

As you can see from the graph below, the Gauss-Laguerre Rule converges most quickly, while the Gauss-Legendre Rule has the worst performance. The intuition behind this result is that the Gauss-Laguerre Rule has an integral boundary that is closest to the original one. As we map the original integral boundary to a wider or narrower boundary that has a significant difference from the original one, the result becomes worse and converges more slowly.

In [34]:
@manipulate for a=-100:1.5:100
    GL_res = zeros(size(3:100)[1])
    GLa_res = zeros(size(3:100)[1])
    GH_res = zeros(size(3:100)[1])
    
    for n=3:100
        GL_res[n-2] = GL_approximation2(a, n=n)
        GLa_res[n-2] = GLa_approximation2(a, n=n)
        GH_res[n-2] = GH_approximation2(a, n=n)
    end
        
    plot(collect(3:100), [GL_res, GLa_res, GH_res], xlabel="number of nodes", ylabel="approximation", label=["Gauss-Legendre" "Gauss-Laguerre" "Gauss-Hermite"])
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["a"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 134, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable{Any}(67), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/chriscyhxx/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/chriscyhxx/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/chriscyhxx/.julia/packages/InteractBase/Qhvxg/src/../assets/all.js"), Asset("css", nothing, "/Users/chriscyhxx/.julia/packages/InteractBase/Qhvxg/src/../assets/style.css"), Asset("css", nothing, "/Users/chriscyhxx/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00000001781f5270, Task (runnable) @0x00000001781f5270), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"-100.0\",\"-98.5\",\"-97.0\",\"-95.5\",\"-94.0\",\"-92.5\",\"-91.0\",\"-89.5\",\"-88.0\",\"-86.5\",\"-85.0\",\"-83.5\",\"-82.0\",\"-80.5\",\"-79.0\",\"-77.5\",\"-76.0\",\"-74.5\",\"-73.0\",\"-71.5\",\"-70.0\",\"-68.5\",\"-67.0\",\"-65.5\",\"-64.0\",\"-62.5\",\"-61.0\",\"-59.5\",\"-58.0\",\"-56.5\",\"-55.0\",\"-53.5